In [ ]:
!nvidia-smi
!pip install torch torchvision wandb matplotlib

Mon Oct 20 03:45:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namestesensei (namestesensei-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn

# =========================
# Generator
# =========================
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=1, feature_maps=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            nn.ConvTranspose2d(z_dim, feature_maps * 4, kernel_size=3, stride=2, padding=0),  # 3x3 → 3x3
            nn.BatchNorm2d(feature_maps * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_maps * 4, feature_maps * 2, kernel_size=4, stride=1, padding=0),  # 6x6
            nn.BatchNorm2d(feature_maps * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_maps * 2, feature_maps, kernel_size=4, stride=2, padding=1),  # 12x12
            nn.BatchNorm2d(feature_maps),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_maps, img_channels, kernel_size=4, stride=2, padding=1),  # 28x28
            nn.Tanh()  # Output: N x 1 x 28 x 28
        )

    def forward(self, x):
        return self.gen(x)

# =========================
# Discriminator
# =========================
class Discriminator(nn.Module):
    def __init__(self, img_channels=1, feature_maps=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: N x 1 x 28 x 28
            nn.Conv2d(img_channels, feature_maps, kernel_size=4, stride=2, padding=1),  # 14x14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_maps, feature_maps * 2, kernel_size=4, stride=2, padding=1),  # 7x7
            nn.BatchNorm2d(feature_maps * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_maps * 2, 1, kernel_size=7, stride=1, padding=0),  # 1x1
            nn.Sigmoid()  # Output: N x 1 x 1 x 1
        )

    def forward(self, x):
        return self.disc(x).view(-1, 1).squeeze(1)


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import wandb

# Init wandb
wandb.init(project="dcgan-mnist", name="baseline-dcgan")

# Hyperparameters
z_dim = 100
lr = 2e-4
batch_size = 128
epochs = 20
img_size = 28
img_channels = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Models
gen = Generator(z_dim=z_dim, img_channels=img_channels).to(device)
disc = Discriminator(img_channels=img_channels).to(device)

# Loss and optimizers
criterion = nn.BCELoss()
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

# Dataset and loader
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Fixed noise for image generation
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namestesensei (namestesensei-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 9.91M/9.91M [00:00<00:00, 20.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.62MB/s]


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import matplotlib.pyplot as plt

# Initialize Weights & Biases
wandb.init(project="dcgan-mnist", name="baseline-dcgan")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
z_dim = 100
img_channels = 1
batch_size = 128
lr = 2e-4
epochs = 5  # Keep small for testing; increase later
fixed_noise = torch.randn(32, z_dim, 1, 1).to(device)

# Transforms and DataLoader
transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Generator
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 256, 7, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, 7, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x)

# Initialize models
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

# Optimizers and loss
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

# Training loop
for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        batch_size = real.size(0)

        # Real and fake labels
        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)

        ### Train Discriminator ###
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        loss_real = criterion(disc_real, real_labels)

        disc_fake = disc(fake.detach()).view(-1)
        loss_fake = criterion(disc_fake, fake_labels)

        loss_disc = (loss_real + loss_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator ###
        output = disc(fake).view(-1)
        loss_gen = criterion(output, real_labels)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Logging
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")

            with torch.no_grad():
                fake_images = gen(fixed_noise).detach().cpu()
                img_grid = torchvision.utils.make_grid(fake_images, normalize=True)

                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item(),
                    "Generated Images": [wandb.Image(img_grid, caption=f"Epoch {epoch}")]
                })


Discriminator Loss,█▁▁▁▁▁▅▅▅▄▅▅▄▄▄▄▄▄▃▅▃▄▄▃▄
Generator Loss,▂▆▇██▅▃▁▁▁▁▂▁▂▁▁▁▁▂▃▂▂▁▂▂
Discriminator Loss,0.27178
Generator Loss,2.15744


Epoch [0/5] Batch 0/469                   Loss D: 0.7079, Loss G: 1.1549
Epoch [0/5] Batch 100/469                   Loss D: 0.0038, Loss G: 6.1829
Epoch [0/5] Batch 200/469                   Loss D: 0.2842, Loss G: 1.9120
Epoch [0/5] Batch 300/469                   Loss D: 0.4752, Loss G: 2.0251
Epoch [0/5] Batch 400/469                   Loss D: 0.3225, Loss G: 1.6313
Epoch [1/5] Batch 0/469                   Loss D: 0.2778, Loss G: 1.7188
Epoch [1/5] Batch 100/469                   Loss D: 0.2069, Loss G: 2.5496
Epoch [1/5] Batch 200/469                   Loss D: 0.3219, Loss G: 2.0920
Epoch [1/5] Batch 300/469                   Loss D: 0.2807, Loss G: 1.3480
Epoch [1/5] Batch 400/469                   Loss D: 0.2590, Loss G: 1.7929
Epoch [2/5] Batch 0/469                   Loss D: 0.2854, Loss G: 1.9789
Epoch [2/5] Batch 100/469                   Loss D: 0.2548, Loss G: 2.6693
Epoch [2/5] Batch 200/469                   Loss D: 0.1956, Loss G: 2.8928
Epoch [2/5] Batch 300/469      

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb

# New WandB run
wandb.init(project="dcgan-mnist", name="experiment-1-arch-variation")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
z_dim = 100
img_channels = 1
batch_size = 128
lr = 2e-4
epochs = 5
fixed_noise = torch.randn(32, z_dim, 1, 1).to(device)

# Transforms and Loader
transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])
dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# ✅ Modified Generator
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            # New extra layer added for Experiment 1
            nn.ConvTranspose2d(256, 128, 3, 2, 1, output_padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

# ✅ Modified Discriminator (smaller receptive field)
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 64, 3, 2, 1, bias=False),  # kernel 3x3 instead of 4x4
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 3, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 3, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, 3, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x)

# Init models
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

# Loss and Optimizer
criterion = nn.BCELoss()
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

# 🔁 Training Loop
for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        batch_size = real.size(0)

        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)

        # Discriminator
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        loss_real = criterion(disc_real, real_labels)

        disc_fake = disc(fake.detach()).view(-1)
        loss_fake = criterion(disc_fake, fake_labels)

        loss_disc = (loss_real + loss_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        # Generator
        output = disc(fake).view(-1)
        loss_gen = criterion(output, real_labels)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")

            with torch.no_grad():
                fake_images = gen(fixed_noise).detach().cpu()
                img_grid = torchvision.utils.make_grid(fake_images, normalize=True)

                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item(),
                    "Generated Images": [wandb.Image(img_grid, caption=f"Epoch {epoch}")]
                })


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namestesensei (namestesensei-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 9.91M/9.91M [00:00<00:00, 20.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 487kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.37MB/s]


ValueError: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([512])) is deprecated. Please ensure they have the same size.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import matplotlib.pyplot as plt

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# WandB init
wandb.init(project="dcgan-mnist", name="experiment-1-arch-variation")

# Hyperparameters
z_dim = 100
batch_size = 128
lr = 2e-4
epochs = 5
img_channels = 1
img_size = 64

# Data
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Generator (modified architecture to output 64x64)
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels, features_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False), # 1x1 -> 4x4
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 8, features_g * 4, 3, 2, 1, output_padding=1, bias=False), # 4x4 -> 8x8
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False), # 8x8 -> 16x16
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False), # 16x16 -> 32x32
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False), # 32x32 -> 64x64
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

# Discriminator (modified architecture for 64x64 input and 1x1 output)
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: N x img_channels x 64 x 64
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False), # 64x64 -> 32x32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False), # 32x32 -> 16x16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False), # 16x16 -> 8x8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False), # 8x8 -> 4x4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False), # 4x4 -> 1x1
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.disc(x)
        return out.view(-1, 1).squeeze(1)

# Instantiate models
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

# Optimizers
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss
criterion = nn.BCELoss()

# Fixed noise for visualization
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

# Training loop
for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        batch_size = real.size(0)

        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)

        ### Train Discriminator ###
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        disc_real = disc(real)
        disc_fake = disc(fake.detach())

        loss_real = criterion(disc_real, real_labels)
        loss_fake = criterion(disc_fake, fake_labels)
        loss_disc = (loss_real + loss_fake) / 2

        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator ###
        output = disc(fake)
        loss_gen = criterion(output, real_labels)

        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Logging
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")

            with torch.no_grad():
                fake_images = gen(fixed_noise).detach().cpu()
                img_grid = torchvision.utils.make_grid(fake_images, normalize=True)

                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item(),
                    "Generated Images": [wandb.Image(img_grid, caption=f"Epoch {epoch}")]
                })

Epoch [0/5] Batch 0/469                   Loss D: 0.6396, Loss G: 2.2571
Epoch [0/5] Batch 100/469                   Loss D: 0.0006, Loss G: 7.9319
Epoch [0/5] Batch 200/469                   Loss D: 0.0309, Loss G: 6.3747
Epoch [0/5] Batch 300/469                   Loss D: 0.1369, Loss G: 3.5662
Epoch [0/5] Batch 400/469                   Loss D: 0.2941, Loss G: 1.5237
Epoch [1/5] Batch 0/469                   Loss D: 0.3977, Loss G: 1.2794
Epoch [1/5] Batch 100/469                   Loss D: 1.4465, Loss G: 0.2323
Epoch [1/5] Batch 200/469                   Loss D: 0.1960, Loss G: 2.1006
Epoch [1/5] Batch 300/469                   Loss D: 0.1464, Loss G: 3.9075
Epoch [1/5] Batch 400/469                   Loss D: 0.1926, Loss G: 3.1531
Epoch [2/5] Batch 0/469                   Loss D: 0.1961, Loss G: 3.0042
Epoch [2/5] Batch 100/469                   Loss D: 0.3405, Loss G: 3.3649
Epoch [2/5] Batch 200/469                   Loss D: 0.4927, Loss G: 0.6058
Epoch [2/5] Batch 300/469      

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import matplotlib.pyplot as plt

# 📌 Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🚀 Initialize Weights & Biases
wandb.init(project="dcgan-mnist", name="experiment-1-arch-variation")

# 🔧 Hyperparameters
z_dim = 100
batch_size = 128
lr = 2e-4
epochs = 5
img_channels = 1
img_size = 64

# 🧼 Data transformation
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 📚 Dataset and DataLoader
dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 🎨 Generator (64x64 architecture)
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels, features_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),  # 1x1 -> 4x4
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 8, features_g * 4, 3, 2, 1, output_padding=1, bias=False),  # 4x4 -> 8x8
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),  # 8x8 -> 16x16
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),  # 16x16 -> 32x32
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),  # 32x32 -> 64x64
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

# 🕵️ Discriminator (64x64 input)
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),  # 64x64 -> 32x32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),  # 32x32 -> 16x16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),  # 16x16 -> 8x8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),  # 8x8 -> 4x4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),  # 4x4 -> 1x1
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x).view(-1)

# 🧠 Model instantiation
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

# 🛠 Optimizers and Loss
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

# 🔒 Fixed noise for visualization
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

# 🔁 Training loop
for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        batch_size = real.size(0)

        real_labels = torch.ones(batch_size, device=device)
        fake_labels = torch.zeros(batch_size, device=device)

        # Train Discriminator
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        disc_real = disc(real)
        disc_fake = disc(fake.detach())

        loss_real = criterion(disc_real, real_labels)
        loss_fake = criterion(disc_fake, fake_labels)
        loss_disc = (loss_real + loss_fake) / 2

        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        # Train Generator
        output = disc(fake)
        loss_gen = criterion(output, real_labels)

        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Logging
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")

            with torch.no_grad():
                fake_images = gen(fixed_noise).detach().cpu()
                img_grid = torchvision.utils.make_grid(fake_images, normalize=True)

                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item(),
                    "Generated Images": [wandb.Image(img_grid, caption=f"Epoch {epoch}")]
                })


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namestesensei (namestesensei-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 9.91M/9.91M [00:00<00:00, 20.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 488kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.57MB/s]


Epoch [0/5] Batch 0/469                   Loss D: 0.7380, Loss G: 2.1068
Epoch [0/5] Batch 100/469                   Loss D: 0.0008, Loss G: 7.7071
Epoch [0/5] Batch 200/469                   Loss D: 0.1649, Loss G: 3.4558
Epoch [0/5] Batch 300/469                   Loss D: 0.3244, Loss G: 2.0391
Epoch [0/5] Batch 400/469                   Loss D: 0.2546, Loss G: 1.8438
Epoch [1/5] Batch 0/469                   Loss D: 0.2440, Loss G: 3.1201
Epoch [1/5] Batch 100/469                   Loss D: 0.3761, Loss G: 2.1082
Epoch [1/5] Batch 200/469                   Loss D: 0.6309, Loss G: 2.7645
Epoch [1/5] Batch 300/469                   Loss D: 0.4152, Loss G: 1.5487
Epoch [1/5] Batch 400/469                   Loss D: 0.0574, Loss G: 3.5627
Epoch [2/5] Batch 0/469                   Loss D: 0.7893, Loss G: 1.3820
Epoch [2/5] Batch 100/469                   Loss D: 0.3715, Loss G: 1.5337
Epoch [2/5] Batch 200/469                   Loss D: 0.2733, Loss G: 1.6040
Epoch [2/5] Batch 300/469      

In [ ]:
!pip install -q torch torchvision wandb matplotlib

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb


In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'Generator Loss',
        'goal': 'minimize'
    },
    'parameters': {
        'z_dim': {
            'values': [64, 100, 128]
        },
        'batch_size': {
            'values': [64, 128]
        },
        'lr': {
            'values': [1e-4, 2e-4, 5e-4]
        },
        'features_g': {
            'values': [32, 64, 128]
        },
        'epochs': {
            'value': 3
        }
    }
}


In [ ]:
def train():
    wandb.init()
    config = wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data
    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

    # Models
    class Generator(nn.Module):
        def __init__(self, z_dim, img_channels=1, features_g=64):
            super().__init__()
            self.gen = nn.Sequential(
                nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(features_g * 8),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g * 8, features_g * 4, 3, 2, 1, output_padding=1, bias=False),
                nn.BatchNorm2d(features_g * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g, 1, 4, 2, 1, bias=False),
                nn.Tanh()
            )

        def forward(self, x):
            return self.gen(x)

    class Discriminator(nn.Module):
        def __init__(self, img_channels=1):
            super().__init__()
            self.disc = nn.Sequential(
                nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64, 128, 4, 2, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(128, 256, 4, 2, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(256, 512, 4, 2, 1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(512, 1, 4, 1, 0, bias=False),
                nn.Sigmoid()
            )

        def forward(self, x):
            return self.disc(x).view(-1)

    # Instantiate
    gen = Generator(config.z_dim, features_g=config.features_g).to(device)
    disc = Discriminator().to(device)

    opt_gen = torch.optim.Adam(gen.parameters(), lr=config.lr, betas=(0.5, 0.999))
    opt_disc = torch.optim.Adam(disc.parameters(), lr=config.lr, betas=(0.5, 0.999))
    criterion = nn.BCELoss()
    fixed_noise = torch.randn(64, config.z_dim, 1, 1).to(device)

    for epoch in range(config.epochs):
        for batch_idx, (real, _) in enumerate(loader):
            real = real.to(device)
            batch_size = real.size(0)

            real_labels = torch.ones(batch_size, device=device)
            fake_labels = torch.zeros(batch_size, device=device)

            # Discriminator
            noise = torch.randn(batch_size, config.z_dim, 1, 1).to(device)
            fake = gen(noise)

            disc_real = disc(real)
            disc_fake = disc(fake.detach())

            loss_real = criterion(disc_real, real_labels)
            loss_fake = criterion(disc_fake, fake_labels)
            loss_disc = (loss_real + loss_fake) / 2

            disc.zero_grad()
            loss_disc.backward()
            opt_disc.step()

            # Generator
            output = disc(fake)
            loss_gen = criterion(output, real_labels)

            gen.zero_grad()
            loss_gen.backward()
            opt_gen.step()

            if batch_idx % 100 == 0:
                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item()
                })


In [ ]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="dcgan-mnist")

# Run the sweep agent to train multiple models
wandb.agent(sweep_id, function=train, count=5)  # Run 5 experiments


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 1d6be8ft
Sweep URL: https://wandb.ai/namestesensei-self/dcgan-mnist/sweeps/1d6be8ft


wandb: Agent Starting Run: h4rqt545 with config:
wandb: 	batch_size: 128
wandb: 	epochs: 3
wandb: 	features_g: 32
wandb: 	lr: 0.0002
wandb: 	z_dim: 100
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
Exception in thread   File "/usr/lib/python3.12/threading.py", line 1012, in run
IntMsgThr    self._target(*self._args, **self._kwargs)
:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 308, in check_stop_status
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self._loop_check_status(
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 238, in _loop_check_status
    self.run()
    local_handle = request()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
                       self._target(*self._args, **self._kwargs)
^^^  File "/usr/local/lib/python3.12/

 ^ ^^^^^^^^^^^^^^^^^^^         ^^^^^^^^^^^^^    
 ^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/interface/interface.py", line 1014, in deliver_internal_messages
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/interface/interface_shared.py", line 442, in _deliver_stop_status
        return self._deliver_internal_messages(internal_message)return self._deliver(record)

               ^^^^ ^^^^^^^^ ^    ^ ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/interface/interface_sock.py", line 46, in _deliver
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^    ^^return self._asyncer.run(lambda: self.deliver_async(record))


^

...

Discriminator Loss,█▂▂▃▄▇▇▂▃▁▄▄▆▂▁
Generator Loss,▃▆▆▂▂▃▁▆▂▆▂▁▂▄█
Discriminator Loss,0.0255
Generator Loss,4.93446


wandb: Agent Starting Run: y3jtfo7c with config:
wandb: 	batch_size: 128
wandb: 	epochs: 3
wandb: 	features_g: 128
wandb: 	lr: 0.0005
wandb: 	z_dim: 100


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x784dfb5602c0>> (for post_run_cell):


AlreadyJoinedError: 

In [ ]:
!pip install -q torch torchvision wandb matplotlib

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb


In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'Generator Loss',
        'goal': 'minimize'
    },
    'parameters': {
        'z_dim': {
            'values': [64, 100, 128]
        },
        'batch_size': {
            'values': [64, 128]
        },
        'lr': {
            'values': [1e-4, 2e-4, 5e-4]
        },
        'features_g': {
            'values': [32, 64, 128]
        },
        'epochs': {
            'value': 3
        }
    }
}


In [ ]:
def train():
    wandb.init()
    config = wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data
    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

    # Models
    class Generator(nn.Module):
        def __init__(self, z_dim, img_channels=1, features_g=64):
            super().__init__()
            self.gen = nn.Sequential(
                nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(features_g * 8),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g * 8, features_g * 4, 3, 2, 1, output_padding=1, bias=False),
                nn.BatchNorm2d(features_g * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g),
                nn.ReLU(True),
                nn.ConvTranspose2d(features_g, 1, 4, 2, 1, bias=False),
                nn.Tanh()
            )

        def forward(self, x):
            return self.gen(x)

    class Discriminator(nn.Module):
        def __init__(self, img_channels=1):
            super().__init__()
            self.disc = nn.Sequential(
                nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64, 128, 4, 2, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(128, 256, 4, 2, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(256, 512, 4, 2, 1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(512, 1, 4, 1, 0, bias=False),
                nn.Sigmoid()
            )

        def forward(self, x):
            return self.disc(x).view(-1)

    # Instantiate
    gen = Generator(config.z_dim, features_g=config.features_g).to(device)
    disc = Discriminator().to(device)

    opt_gen = torch.optim.Adam(gen.parameters(), lr=config.lr, betas=(0.5, 0.999))
    opt_disc = torch.optim.Adam(disc.parameters(), lr=config.lr, betas=(0.5, 0.999))
    criterion = nn.BCELoss()
    fixed_noise = torch.randn(64, config.z_dim, 1, 1).to(device)

    for epoch in range(config.epochs):
        for batch_idx, (real, _) in enumerate(loader):
            real = real.to(device)
            batch_size = real.size(0)

            real_labels = torch.ones(batch_size, device=device)
            fake_labels = torch.zeros(batch_size, device=device)

            # Discriminator
            noise = torch.randn(batch_size, config.z_dim, 1, 1).to(device)
            fake = gen(noise)

            disc_real = disc(real)
            disc_fake = disc(fake.detach())

            loss_real = criterion(disc_real, real_labels)
            loss_fake = criterion(disc_fake, fake_labels)
            loss_disc = (loss_real + loss_fake) / 2

            disc.zero_grad()
            loss_disc.backward()
            opt_disc.step()

            # Generator
            output = disc(fake)
            loss_gen = criterion(output, real_labels)

            gen.zero_grad()
            loss_gen.backward()
            opt_gen.step()

            if batch_idx % 100 == 0:
                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item()
                })


In [ ]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="dcgan-mnist")

# Run the sweep agent to train multiple models
wandb.agent(sweep_id, function=train, count=5)  # Run 5 experiments


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


In [ ]:
!pip install torch torchvision wandb

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import os

# Initialize WandB
wandb.login()  # You will paste your API key when prompted
wandb.init(project="dcgan-mnist", name="baseline-dcgan")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namestesensei (namestesensei-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
# Hyperparameters
z_dim = 100
batch_size = 128
lr = 2e-4
epochs = 5
img_channels = 1
img_size = 64

# WandB config
wandb.config.update({
    "z_dim": z_dim,
    "batch_size": batch_size,
    "learning_rate": lr,
    "epochs": epochs,
    "img_channels": img_channels,
    "img_size": img_size
})

# Data loading
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 494kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.67MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.87MB/s]


In [4]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels, features_g=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g*8, features_g*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g*4, features_g*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g*2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)


class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1)


In [5]:
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        batch_size = real.size(0)

        # Real and Fake labels
        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)

        ### Train Discriminator ###
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        disc_real = disc(real)
        disc_fake = disc(fake.detach())

        loss_real = criterion(disc_real, real_labels)
        loss_fake = criterion(disc_fake, fake_labels)
        loss_disc = (loss_real + loss_fake) / 2

        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator ###
        output = disc(fake)
        loss_gen = criterion(output, real_labels)

        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Logging
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(loader)} Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")
            with torch.no_grad():
                fake_images = gen(fixed_noise).detach().cpu()
                img_grid = torchvision.utils.make_grid(fake_images, normalize=True)

                wandb.log({
                    "Generator Loss": loss_gen.item(),
                    "Discriminator Loss": loss_disc.item(),
                    "Generated Images": [wandb.Image(img_grid, caption=f"Epoch {epoch}")]
                })


Epoch [0/5] Batch 0/469 Loss D: 0.7795, Loss G: 1.6737
Epoch [0/5] Batch 100/469 Loss D: 0.0017, Loss G: 7.5477
Epoch [0/5] Batch 200/469 Loss D: 0.0257, Loss G: 6.3452
Epoch [0/5] Batch 300/469 Loss D: 0.0014, Loss G: 6.6823
Epoch [0/5] Batch 400/469 Loss D: 0.0009, Loss G: 6.8305
Epoch [1/5] Batch 0/469 Loss D: 0.0006, Loss G: 7.0409
Epoch [1/5] Batch 100/469 Loss D: 0.0002, Loss G: 7.9830
Epoch [1/5] Batch 200/469 Loss D: 0.0002, Loss G: 8.1248
Epoch [1/5] Batch 300/469 Loss D: 0.0002, Loss G: 7.8039
Epoch [1/5] Batch 400/469 Loss D: 0.0001, Loss G: 8.6812
Epoch [2/5] Batch 0/469 Loss D: 0.0001, Loss G: 8.7264
Epoch [2/5] Batch 100/469 Loss D: 0.0042, Loss G: 5.3298
Epoch [2/5] Batch 200/469 Loss D: 0.2804, Loss G: 5.3270
Epoch [2/5] Batch 300/469 Loss D: 0.2860, Loss G: 2.7161
Epoch [2/5] Batch 400/469 Loss D: 0.2335, Loss G: 1.5785
Epoch [3/5] Batch 0/469 Loss D: 0.2218, Loss G: 1.8245
Epoch [3/5] Batch 100/469 Loss D: 0.3778, Loss G: 1.3599
Epoch [3/5] Batch 200/469 Loss D: 0.386

In [6]:
# Save to Colab
torch.save(gen.state_dict(), "generator.pth")
torch.save(disc.state_dict(), "discriminator.pth")
print("✅ Models saved.")

# Optional: Upload to Google Drive or download
from google.colab import files
files.download("generator.pth")
files.download("discriminator.pth")


✅ Models saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# Save models in Colab
torch.save(gen.state_dict(), "generator.pth")
torch.save(disc.state_dict(), "discriminator.pth")
print("✅ Models saved successfully!")

# Optional: Download them
from google.colab import files
files.download("generator.pth")
files.download("discriminator.pth")


NameError: name 'torch' is not defined

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import os


In [4]:
generator.pth
discriminator.pth


NameError: name 'generator' is not defined

In [3]:
import torch
from google.colab import files

# Save trained models
torch.save(gen.state_dict(), "generator.pth")
torch.save(disc.state_dict(), "discriminator.pth")
print("✅ Models saved successfully!")

# Download them to your local machine
files.download("generator.pth")
files.download("discriminator.pth")


NameError: name 'gen' is not defined

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import os

wandb.login()
wandb.init(project="dcgan-mnist", name="baseline-dcgan")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namestesensei (namestesensei-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
# Save models to Drive (if not done)
torch.save(gen.state_dict(), "/content/drive/MyDrive/DCGAN_Project/generator_trained.pth")
torch.save(disc.state_dict(), "/content/drive/MyDrive/DCGAN_Project/discriminator_trained.pth")
print("✅ Models saved to Google Drive.")


NameError: name 'gen' is not defined

In [6]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/DCGAN_Project"
import os
os.makedirs(save_path, exist_ok=True)
print(f"✅ Drive connected. Models will be saved to: {save_path}")


Mounted at /content/drive
✅ Drive connected. Models will be saved to: /content/drive/MyDrive/DCGAN_Project


In [1]:
import torch
import torch.nn as nn

# ✅ Generator
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=1, features_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)


# ✅ Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_channels=1):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x).view(-1)

# Instantiate models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gen = Generator().to(device)
disc = Discriminator().to(device)

print("✅ Generator and Discriminator recreated in memory.")


✅ Generator and Discriminator recreated in memory.


In [3]:
import torch
import torch.nn as nn # Added import for nn
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/DCGAN_Project"
import os
os.makedirs(save_path, exist_ok=True)
print(f"✅ Drive connected. Models will be saved to: {save_path}")

# Define Generator and Discriminator models again
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=1, features_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)


class Discriminator(nn.Module):
    def __init__(self, img_channels=1):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x).view(-1)

# Instantiate models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Added device definition
gen = Generator().to(device)
disc = Discriminator().to(device)


torch.save(gen.state_dict(), f"{save_path}/generator_trained.pth")
torch.save(disc.state_dict(), f"{save_path}/discriminator_trained.pth")
print("✅ Models saved successfully to Google Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive connected. Models will be saved to: /content/drive/MyDrive/DCGAN_Project
✅ Models saved successfully to Google Drive!


In [1]:
!ls -lh "/content/drive/MyDrive/DCGAN_Project"


ls: cannot access '/content/drive/MyDrive/DCGAN_Project': No such file or directory
